In [1]:
import os
# pyspark --packages graphframes:graphframes:0.3.0-spark2.0-s_2.11 --jars graphframes-0.3.0-spark2.0-s_2.11.jar
pyspark_submit_args = '--packages graphframes:graphframes-0.8.0-spark3.0-s_2.12 pyspark-shell'# --jars graphframes-0.8.0-spark3.0-s_2.12.jar'
pyspark_submit_args = '--jars graphframes-0.8.0-spark3.0-s_2.12.jar pyspark-shell'

os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args
import findspark
findspark.init()
findspark.find()
import pyspark
conf = pyspark.SparkConf()
sc = pyspark.SparkContext(conf=conf)

# from pyspark import *


In [2]:
from pyspark import *
from pyspark.sql import *
from graphframes import *
sparkSession = SparkSession.builder.appName('A4').master('local').getOrCreate()
# spark = SparkSession.builder.appName('BDA_A2').master('local').getOrCreate()

In [3]:
dataset = open('soc-Epinions1.txt', 'r')
lines = dataset.readlines()
lines = lines[4:] # skipping the comments in file

In [4]:
vertices = set()
edges = []
for line in lines:
    source, dest = line.split()
    vertices.add((source,))
    vertices.add((dest,))
    edges.append((source, dest))
#     print(source, dest)
#     break
vertices = list(vertices)

In [5]:
vertices_df = sparkSession.createDataFrame(vertices, ["id"])
edges_df = sparkSession.createDataFrame(edges, ["src", "dst"])

Py4JJavaError: An error occurred while calling o26.applySchemaToPythonRDD.
: java.lang.NoClassDefFoundError: scala/Product$class
	at scala.reflect.internal.Symbols$SymbolOps.<init>(Symbols.scala:3631)
	at scala.reflect.internal.Symbols$class.$init$(Symbols.scala:3632)
	at scala.reflect.internal.SymbolTable.<init>(SymbolTable.scala:16)
	at scala.reflect.runtime.JavaUniverse.<init>(JavaUniverse.scala:16)
	at scala.reflect.runtime.package$.universe$lzycompute(package.scala:17)
	at scala.reflect.runtime.package$.universe(package.scala:17)
	at org.apache.spark.sql.catalyst.ScalaReflection$.<init>(ScalaReflection.scala:48)
	at org.apache.spark.sql.catalyst.ScalaReflection$.<clinit>(ScalaReflection.scala)
	at org.apache.spark.sql.catalyst.encoders.RowEncoder$.serializerFor(RowEncoder.scala:77)
	at org.apache.spark.sql.catalyst.encoders.RowEncoder$.apply(RowEncoder.scala:66)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:88)
	at org.apache.spark.sql.SparkSession.internalCre

In [6]:
graph = GraphFrame(vertices_df, edges_df)

In [7]:
graph.inDegrees.show()
alpha = 0.8

+----+--------+
|  id|inDegree|
+----+--------+
|1572|     180|
|2069|      51|
| 296|     121|
|1512|     131|
|2904|      53|
|3414|      16|
| 829|      63|
| 675|      33|
|2088|      80|
|2162|      61|
|4032|      17|
|2294|      56|
|6613|      59|
|6731|       8|
|1436|     181|
| 691|      47|
|9009|      34|
|3210|      34|
|9586|       1|
|9993|       2|
+----+--------+
only showing top 20 rows



In [8]:
results = graph.pageRank(resetProbability=1-alpha, maxIter=20)
results.vertices.sort('pagerank' , ascending=False).show()

+----+------------------+
|  id|          pagerank|
+----+------------------+
|  18|325.88530464894075|
| 737| 212.6657655504572|
|1719|147.22687135149994|
| 118|145.06701914713187|
| 790| 142.3344648404173|
| 136|139.79117216951684|
| 143| 139.0543177981888|
|  40|118.38451283685471|
|1619|106.36353372544302|
|4415|101.46022715498134|
|1179| 98.47148432038729|
| 849|   98.249965640328|
| 725| 98.18624344474455|
|  27| 95.18549998640358|
|1401| 93.16161309296348|
|  77| 92.67561681223977|
| 128| 92.61946884069384|
| 401| 92.60553359356562|
|1191| 92.21772965271713|
| 135| 89.42293162565127|
+----+------------------+
only showing top 20 rows



In [12]:
# https://graphframes.github.io/graphframes/docs/_site/api/python/graphframes.html#graphframes.GraphFrame.parallelPersonalizedPageRank
results2 = graph.pageRank(resetProbability=1-alpha, maxIter=20, sourceId="18")


In [13]:
results2.vertices.sort('pagerank' , ascending=False).show()

+----+--------------------+
|  id|            pagerank|
+----+--------------------+
|  18| 0.21409061657990314|
| 118|0.006807206592250928|
| 790|0.006075429899075042|
| 136|0.005978881253253704|
|1191|0.005723130330628862|
| 128|0.005716570772041351|
|  59|0.005452936546152758|
|1909|0.005371970954204318|
| 735|0.005201034886617...|
|1398|0.005071498194091262|
|1910|0.005065724611604955|
| 848|0.004906769572804259|
|1903|0.004875843633775832|
|1894|0.004832714557261...|
|1897|0.004739363135899272|
| 722|0.004714263330756526|
|1905|0.004637281564599612|
|1399|0.004628715582848625|
|1754|0.004615841362154648|
|1716| 0.00456015521041058|
+----+--------------------+
only showing top 20 rows

